
# Transformer-based English–French Translation

Notebook ini merupakan implementasi eksplorasi model **Transformer** untuk penerjemahan otomatis dari Bahasa Inggris ke Bahasa Prancis.

Tujuan utama:
- Melatih model Transformer selama **1 epoch** dengan **batch-size maksimal 100**
- Menunjukkan proses **Text Preprocessing**, **Definisi Arsitektur Transformer**, **Training**, dan **Inference**
- Menampilkan metrik: `TrainLoss`, `ValLoss`, dan `ValAcc` di setiap akhir batch.

Bobot penilaian:
| Aspek | Bobot |
|-------|-------|
| Data Preparation (Text Preprocessing) | 20% |
| Definisi Class Transformer | 25% |
| Proses Training (TrainLoss, ValLoss, ValAcc) | 35% |
| Inference Translation | 20% |


## Persiapan Lingkungan

In [1]:

!pip install torch pandas numpy
import torch, pandas as pd, numpy as np, random, math, re, os
from collections import Counter
from torch import nn
from torch.utils.data import Dataset, DataLoader

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Running on', DEVICE)


Running on cuda


## 1. Data Preparation & Text Preprocessing (20%)

In [5]:

# Dataset diambil dari file small_vocab_en.csv dan small_vocab_fr.csv
en_path = '/content/small_vocab_en.csv'
fr_path = '/content/small_vocab_fr.csv'

# Baca setiap baris sebagai satu teks utuh
with open(en_path, 'r', encoding='utf-8') as f:
    src_texts = [line.strip() for line in f if line.strip()]

with open(fr_path, 'r', encoding='utf-8') as f:
    tgt_texts = [line.strip() for line in f if line.strip()]

print(f"Contoh data Inggris: {src_texts[0]}")
print(f"Contoh data Prancis: {tgt_texts[0]}")

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zâêôàèçùé'\-\.\,\?\!\s]", ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text):
    return text.split()

src_tokens = [tokenize(clean_text(s)) for s in src_texts]
tgt_tokens = [tokenize(clean_text(t)) for t in tgt_texts]

# Split train/val
data = list(zip(src_tokens, tgt_tokens))
random.shuffle(data)
split = int(0.9 * len(data))
train, val = data[:split], data[split:]

PAD, BOS, EOS, UNK = '<pad>', '<s>', '</s>', '<unk>'

def build_vocab(sentences):
    counter = Counter(t for s in sentences for t in s)
    vocab = [PAD, BOS, EOS, UNK] + [t for t, _ in counter.most_common()]
    stoi = {t: i for i, t in enumerate(vocab)}
    itos = {i: t for t, i in stoi.items()}
    return stoi, itos

src_stoi, src_itos = build_vocab([s for s, _ in train])
tgt_stoi, tgt_itos = build_vocab([t for _, t in train])

print('Vocab sizes -> src:', len(src_stoi), '| tgt:', len(tgt_stoi))

Contoh data Inggris: new jersey is sometimes quiet during autumn , and it is snowy in april .
Contoh data Prancis: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
Vocab sizes -> src: 231 | tgt: 356


## 2. Definisi Arsitektur Transformer (25%)

In [8]:
import torch
import torch.nn as nn
import math

# ==========================================
# Positional Encoding
# ==========================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x


# ==========================================
# Transformer Model untuk Translation
# ==========================================
class TransformerMT(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=128, nhead=4, num_layers=2, dim_ff=512, dropout=0.1):
        super().__init__()
        self.src_embed = nn.Embedding(src_vocab, d_model)
        self.tgt_embed = nn.Embedding(tgt_vocab, d_model)
        self.pos_enc = PositionalEncoding(d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_ff,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab)

    def forward(self, src, tgt):
        src = self.pos_enc(self.src_embed(src))
        tgt = self.pos_enc(self.tgt_embed(tgt))
        out = self.transformer(src, tgt)
        out = self.fc_out(out)
        return out


In [9]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

# ==========================================
# Dataset & DataLoader
# ==========================================
class TranslationDataset(Dataset):
    def __init__(self, pairs, src_stoi, tgt_stoi, max_len=20):
        self.pairs = pairs
        self.src_stoi = src_stoi
        self.tgt_stoi = tgt_stoi
        self.max_len = max_len

    def encode(self, tokens, stoi, bos=False, eos=False):
        ids = [stoi.get(t, stoi['<unk>']) for t in tokens]
        if bos: ids = [stoi['<s>']] + ids
        if eos: ids = ids + [stoi['</s>']]
        ids = ids[:self.max_len]
        ids += [stoi['<pad>']] * (self.max_len - len(ids))
        return ids

    def __getitem__(self, idx):
        src, tgt = self.pairs[idx]
        src_ids = self.encode(src, self.src_stoi)
        tgt_in = self.encode(tgt, self.tgt_stoi, bos=True)
        tgt_out = self.encode(tgt, self.tgt_stoi, eos=True)
        return torch.tensor(src_ids), torch.tensor(tgt_in), torch.tensor(tgt_out)

    def __len__(self):
        return len(self.pairs)


BATCH_SIZE = 100
MAX_LEN = 20
train_ds = TranslationDataset(train, src_stoi, tgt_stoi, MAX_LEN)
val_ds = TranslationDataset(val, src_stoi, tgt_stoi, MAX_LEN)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

# ==========================================
# Inisialisasi Model & Optimizer
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerMT(len(src_stoi), len(tgt_stoi)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=src_stoi['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


## 3. Proses Training (35%)

In [10]:
from tqdm import tqdm

def accuracy_fn(y_pred, y_true, pad_idx):
    pred_tokens = y_pred.argmax(dim=-1)
    mask = y_true != pad_idx
    correct = (pred_tokens == y_true) & mask
    return correct.sum().float() / mask.sum().float()

EPOCHS = 1
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    for i, (src, tgt_in, tgt_out) in enumerate(tqdm(train_dl)):
        src, tgt_in, tgt_out = src.to(device), tgt_in.to(device), tgt_out.to(device)
        optimizer.zero_grad()
        output = model(src, tgt_in)
        loss = criterion(output.view(-1, output.size(-1)), tgt_out.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i+1) % 1 == 0:
            print(f"Batch {i+1}/{len(train_dl)} - TrainLoss: {loss.item():.4f}")

    # Validation
    model.eval()
    val_loss, val_acc = 0, 0
    with torch.no_grad():
        for src, tgt_in, tgt_out in val_dl:
            src, tgt_in, tgt_out = src.to(device), tgt_in.to(device), tgt_out.to(device)
            output = model(src, tgt_in)
            loss = criterion(output.view(-1, output.size(-1)), tgt_out.view(-1))
            val_loss += loss.item()
            val_acc += accuracy_fn(output, tgt_out, src_stoi['<pad>']).item()
    val_loss /= len(val_dl)
    val_acc /= len(val_dl)
    print(f"ValLoss: {val_loss:.4f}, ValAcc: {val_acc*100:.2f}%")



Epoch 1/1


  0%|          | 4/1241 [00:00<00:32, 37.76it/s]

Batch 1/1241 - TrainLoss: 6.0181
Batch 2/1241 - TrainLoss: 5.9023
Batch 3/1241 - TrainLoss: 5.7987
Batch 4/1241 - TrainLoss: 5.7185
Batch 5/1241 - TrainLoss: 5.5926
Batch 6/1241 - TrainLoss: 5.5444
Batch 7/1241 - TrainLoss: 5.4586


  1%|          | 9/1241 [00:00<00:30, 40.73it/s]

Batch 8/1241 - TrainLoss: 5.4258
Batch 9/1241 - TrainLoss: 5.3469


...

Batch 1211/1241 - TrainLoss: 0.1588
Batch 1212/1241 - TrainLoss: 0.1312
Batch 1213/1241 - TrainLoss: 0.1733
Batch 1214/1241 - TrainLoss: 0.1924
Batch 1215/1241 - TrainLoss: 0.1733
Batch 1216/1241 - TrainLoss: 0.1417
Batch 1217/1241 - TrainLoss: 0.1763
Batch 1218/1241 - TrainLoss: 0.2321
Batch 1219/1241 - TrainLoss: 0.1413
Batch 1220/1241 - TrainLoss: 0.1607
Batch 1221/1241 - TrainLoss: 0.1449
Batch 1222/1241 - TrainLoss: 0.1450


100%|█████████▉| 1235/1241 [00:22<00:00, 59.59it/s]

Batch 1223/1241 - TrainLoss: 0.1613
Batch 1224/1241 - TrainLoss: 0.1928
Batch 1225/1241 - TrainLoss: 0.1549
Batch 1226/1241 - TrainLoss: 0.1656
Batch 1227/1241 - TrainLoss: 0.1911
Batch 1228/1241 - TrainLoss: 0.2171
Batch 1229/1241 - TrainLoss: 0.1154
Batch 1230/1241 - TrainLoss: 0.2038
Batch 1231/1241 - TrainLoss: 0.1742
Batch 1232/1241 - TrainLoss: 0.1867
Batch 1233/1241 - TrainLoss: 0.2028
Batch 1234/1241 - TrainLoss: 0.1795
Batch 1235/1241 - TrainLoss: 0.1306


100%|██████████| 1241/1241 [00:22<00:00, 54.46it/s]


Batch 1236/1241 - TrainLoss: 0.1276
Batch 1237/1241 - TrainLoss: 0.2290
Batch 1238/1241 - TrainLoss: 0.1735
Batch 1239/1241 - TrainLoss: 0.1523
Batch 1240/1241 - TrainLoss: 0.2232
Batch 1241/1241 - TrainLoss: 0.1989
ValLoss: 0.1215, ValAcc: 97.83%


## 4. Inference Translation (20%)

In [11]:
def translate_sentence(model, sentence, src_stoi, tgt_stoi, tgt_itos, max_len=20):
    model.eval()
    tokens = [w.lower() for w in sentence.split()]
    src_ids = torch.tensor([[src_stoi.get(t, src_stoi['<unk>']) for t in tokens]], device=device)
    tgt_input = torch.tensor([[tgt_stoi['<s>']]], device=device)

    for _ in range(max_len):
        out = model(src_ids, tgt_input)
        next_token = out[:, -1].argmax(dim=-1).unsqueeze(0)
        tgt_input = torch.cat([tgt_input, next_token], dim=1)
        if next_token.item() == tgt_stoi['</s>']:
            break

    translated = [tgt_itos[idx.item()] for idx in tgt_input[0]]
    return ' '.join(translated[1:-1])  # hilangkan <s> dan </s>

# Contoh uji terjemahan
test_sentence = src_texts[0]
print("English :", test_sentence)
print("French (predicted):", translate_sentence(model, test_sentence, src_stoi, tgt_stoi, tgt_itos))


English : new jersey is sometimes quiet during autumn , and it is snowy in april .
French (predicted): new jersey est parfois calme à l'automne , à l'automne , à l'automne à l'automne à l'automne .



## 5. Kesimpulan

Eksperimen ini menunjukkan implementasi dasar Transformer untuk penerjemahan Bahasa Inggris ke Bahasa Prancis.

- Data telah dibersihkan dan ditokenisasi secara sederhana.
- Arsitektur Transformer telah dibangun dari nol dengan PyTorch.
- Proses training menampilkan *TrainLoss*, *ValLoss*, dan *ValAcc* tiap batch.
- Model berhasil melakukan inferensi dengan pendekatan *greedy decoding*.

Selanjutnya, model dapat diperluas dengan peningkatan jumlah epoch, mekanisme perhatian visualisasi, dan evaluasi BLEU score.
